# 04 – Transformación: videos_stactic

Este notebook transforma el JSON crudo del recurso `videos`
en una tabla etática que se actualiza y reescribe cada vez que se consulta

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

RAW_PATH = PROJECT_ROOT / "data" / "raw" / "youtube"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

RAW_PATH, PROCESSED_PATH

In [ ]:
import json
import pandas as pd
from datetime import datetime, date
import isodate

In [ ]:
with open(RAW_PATH / "videos.json", "r", encoding="utf-8") as f:
    videos_raw = json.load(f)

len(videos_raw["items"])


In [ ]:
from datetime import datetime, timezone

EXTRACTED_AT = datetime.now(timezone.utc)

EXTRACTED_AT

In [ ]:
import isodate

def iso_duration_to_seconds(duration):
    try:
        return int(isodate.parse_duration(duration).total_seconds())
    except Exception:
        return None


In [ ]:
rows = []

for item in videos_raw["items"]:

    thumbnails = item["snippet"].get("thumbnails", {})

    thumbnail_url = (
        thumbnails.get("maxres", {}).get("url") or
        thumbnails.get("standard", {}).get("url") or
        thumbnails.get("high", {}).get("url") or
        thumbnails.get("medium", {}).get("url") or
        thumbnails.get("default", {}).get("url")
    )

    video_url = f"https://www.youtube.com/watch?v={item['id']}"

    rows.append({
        "video_id": item["id"],
        "channel_id": item["snippet"]["channelId"],
        "title": item["snippet"]["title"],
        "description": item["snippet"].get("description"),
        "category_id": item["snippet"].get("categoryId"),
        "published_at": pd.to_datetime(
            item["snippet"]["publishedAt"],
            utc=True
        ),
        "duration_seconds": iso_duration_to_seconds(
            item["contentDetails"].get("duration")
        ),
        "thumbnail_url": thumbnail_url,
        "video_url": video_url,
        "extracted_at": EXTRACTED_AT
    })

df_videos_static = pd.DataFrame(rows)
df_videos_static.head()



In [ ]:
df_videos_static.shape

In [ ]:
df_videos_static.dtypes

In [ ]:
#Ordenar columnas
df_videos_static = df_videos_static[
    [
        "video_id",
        "channel_id",
        "title",
        "description",
        "category_id",
        "published_at",
        "duration_seconds",
        "thumbnail_url",
        "video_url",
        "extracted_at"
    ]
]


> ⚠️ Nota (fase de desarrollo)
>
> El guardado en formato Parquet se utiliza temporalmente para pruebas y separación entre notebooks.
> En la versión final del pipeline (.py), este paso será omitido y el DataFrame se enviará directamente a BigQuery, donde residirá el histórico definitivo.

In [ ]:
output_file = PROCESSED_PATH / "videos_static.parquet"

df_videos_static.to_parquet(output_file, index=False)

output_file


In [ ]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
file_path = PROJECT_ROOT / "data" / "processed" / "youtube" / "videos_static.parquet"

df = pd.read_parquet(file_path)
df